# Lecture 07. Fun with filters: let's play some music!

## Instructions

Please work through this activity together in groups of 3-4. Along the way there will be code blocks to fill in, and conceptual questions to answer. Once you have completed all the sections, you have the **option** to submit (as a group) one copy of this worksheet, and a selected audio sample in PrairieLearn.

**This is not a mandatory assignment**. Instead, contributors to each submission will be granted up to 2 bonus points (points, not %) that can be "banked" and later applied to any assignment, quiz, or test during the term. To earn the full two points, complete all the functions and answer all the conceptual questions (unless marked optional). Partially-complete submissions will earn 1 point.

**Submission deadline**: Sunday at 23:59pm. Late submissions will not be considered for bonus points (but, I will still listen to your work!).

Please use the PrairieLearn group work functionality to establish your groups.

## Overview

In Tuesday's lecture, we saw some examples of filters:
 - frequency-selective filters, such as highpass, lowpass, and bandpass, that eliminate or strongly attenuate certain frequencies
 - frequency-shaping filters, that can adjust the relative strengths (gain) of different parts of the frequency spectrum

A common application of these techniques is in audio processing and equalization. With the right filters, we can boost the bass in a song, remove certain instruments, add distortion, and even change the musical key.

All of this is made possible by an algorithm called the fast Fourier *transform* (FFT). This efficient algorithm is at the core of modern-day signal processing, and later in the course we will spend a couple lectures covering it in detail. The goal of this activity is to give you a taste of what you can do with it, and show how the material we're learning in the course can be applied in every day life. After working through this notebook, you'll be able to:
 - describe how the process of music equalization works from a signals and systems perspective
 - apply the FFT and related methods to a signal using Python and NumPy
 - use the FFT to manipulate and modify audio signals

The applications of this go beyond just playing with music. You can imagine using a similar process to remove noise from an audio sample in order to make it clearer, remove the sounds of someone typing on a keyboard during a video call, or even (as we will see later in the course) processing other types of signals such as images.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from IPython.display import Audio, Image
from IPython.core.display import HTML

## Part 1: import your music

Prior to this lecture, I asked you to produce a short clip of your favourite song as a `.wav` file. Below is some code that will should load it up and transform it into a NumPy array. Depending on how you extracted your audio sample (e.g., number of audio channels), you may need to adjust some of the code.

In [ ]:
input_audio = wavfile.read("/content/Elec221Song.wav") # Change your filename here

# The result of importing a wavfile should be a tuple of two elements:
# the sample rate, and the audio signal, which may be have multiple channels.
sample_rate = input_audio[0]

# My audio sample had multiple channels. Yours may not.
# In my case, I took just one of these channels, and converted it from integer values to floats.
channel_0 = input_audio[1][:, 0]
audio = np.array(channel_0, dtype=np.float64) / np.max(channel_0)

If all goes well in the previous cell, you should be able to run the following in order to hear your audio in the Jupyter notebook.

In [ ]:
Audio(audio, rate=sample_rate)

**Question 1a**: do you think we are working in discrete time, or continuous time here?

**Question 1b**: how does the audio from the `.wav` file sound compared to whatever platform you normally use to listen to music? What do you think has happened to it?

Now, let's plot your audio signal.

In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(audio)

Most likely, you will see that there is a LOT going on here. But using Fourier analysis, we can pick out and play with the important stuff.

## Part 2: analyze the spectrum

In this cell, we will extract the frequency spectrum (i.e., the Fourier coefficients), of our song. We'll do a few things here that will become clearer next week. First, we will apply a function called `np.fft.rfft`. The `r` stands for real, and we are using it here because our signal is real-valued. Then, we will use a function called `np.fft.rfftfreqs`, which will provide for us the "real life" frequency values (in Hz) corresponding to each of the coefficients, rather than integer values.

In [ ]:
# A note on normalization: in class, we expressed our regular signal as x[n] = \sum_k c_k exp[2πjkn/N]
# and the Fourier coefficients as c_k = (1/N) \sum_n x[n] exp[-2πjkn/N]. However, the NumPy FFT
# functions have the factor of 1/N on the signal part, rather than on the coefficients. This is
# just a difference in definition and convention; in order to match the way we have been doing things
# in class, we will set norm="forward".
fourier_coefficients = np.fft.rfft(audio, norm="forward")
frequency_spectrum =  np.fft.rfftfreq(len(audio), 1 / sample_rate)

Let's now plot the coefficients. Recall that they are in general complex; we'll plot only the real part for now.

In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(frequency_spectrum, fourier_coefficients.real)
plt.xlabel("Frequency (Hz)", fontsize=14)

**Question 2a**: In what frequency range does most of the signal lie? Does this make sense?

**Question 2b (optional)**: For those of you with musical training, take a closer look at the first ~2000Hz or so of the frequency spectrum. Where do you see peaks, and what does that tell you about the music details of your song?

## Part 3: frequency-selective filters

Now that we have our frequency spectrum, we can can start manipulating our music. Let's start with the basics: complete the functions with the appropriate frequency response to implement simple lowpass, highpass, bandpass, and bandstop filters.

In [ ]:
def lowpass_filter(frequencies, spectrum, max_freq):
    """A lowpass filter.

    Filter out the parts of the spectrum above the specified frequency.

    Args:
        frequencies (array[float]): The frequencies of the Fourier coefficients
            (in Hz, obtained from np.fft.rfftfreqs).
        spectrum (array[complex]): The Fourier coefficients obtained by applying
            the FFT to an audio signal.
        max_freq (float): The maximum frequency, in Hz, allowed by the lowpass filter.

    Returns:
        array[complex]: A modified spectrum containing only the Fourier coefficients
        up to the specified frequency.
    """
    output = []
    for i in range (0, len(frequencies)):
      if frequencies[i] < max_freq:
        output.append(spectrum[i])
      else:
        output.append(0)
    return output


def highpass_filter(frequencies, spectrum, min_freq):
    """A highpass filter.

    Filter out the parts of the spectrum below the specified frequency.

    Args:
        frequencies (array[float]): The frequencies of the Fourier coefficients
            (in Hz, obtained from np.fft.rfftfreqs).
        spectrum (array[complex]): The Fourier coefficients obtained by applying
            the FFT to an audio signal.
        min_freq (float): The minimum frequency, in Hz, allowed by the highpass filter.

    Returns:
        array[complex]: A modified spectrum containing only the Fourier coefficients
        above the specified frequency.
    """
    output = []
    for i in range (0, len(frequencies)):
      if frequencies[i] > min_freq:
        output.append(spectrum[i])
      else:
        output.append(0)
    return output


def bandpass_filter(frequencies, spectrum, min_freq, max_freq):
    """A bandpass filter.

    Keep only the parts of the frequency spectrum in a given range.

    Args:
        frequencies (array[float]): The frequencies of the Fourier coefficients
            (in Hz, obtained from np.fft.rfftfreqs).
        spectrum (array[complex]): The Fourier coefficients obtained by applying
            the FFT to an audio signal.
        min_freq (float): The minimum frequency, in Hz, allowed by the bandpass filter.
        max_freq (float): The maximum frequency, in Hz, allowed by the bandpass filter.

    Returns:
        array[complex]: A modified spectrum containing only the Fourier coefficients
        within the given frequency band.
    """
    output = []
    for i in range (0, len(frequencies)):
      if frequencies[i] < max_freq and frequencies[i] > min_freq:
        output.append(spectrum[i])
      else:
        output.append(0)
    return output


def bandstop_filter(frequencies, spectrum, min_freq, max_freq):
    """A bandstop filter.

    Remove parts of the frequency spectrum in a given range.

    Args:
        frequencies (array[float]): The frequencies of the Fourier coefficients
            (in Hz, obtained from np.fft.rfftfreqs).
        spectrum (array[complex]): The Fourier coefficients obtained by applying
            the FFT to an audio signal.
        min_freq (float): The lowest frequency, in Hz, stopped by the filter.
        max_freq (float): The highest frequency, in Hz, stopped by the filter.

    Returns:
        array[complex]: A modified spectrum containing only the Fourier coefficients
        outside the given frequency band.
    """
    output = []
    for i in range (0, len(frequencies)):
      if not (frequencies[i] < max_freq and frequencies[i] > min_freq):
        output.append(spectrum[i])
      else:
        output.append(0)
    return output

Try applying some of your filters to your Fourier coefficients. Plot your results to see what happened to the spectrum to make sure they are working as expected.

In [ ]:
fourier_coefficients = np.fft.rfft(audio, norm="forward")
frequency_spectrum =  np.fft.rfftfreq(len(audio), 1 / sample_rate)

lowpass_song = lowpass_filter(frequency_spectrum,fourier_coefficients,2500)

plt.figure(figsize=(20, 5))
plt.plot(frequency_spectrum,lowpass_song)
plt.xlabel("Frequency (Hz)", fontsize=14)

Audio(np.fft.irfft(lowpass_song), rate=sample_rate)

In [ ]:
fourier_coefficients = np.fft.rfft(audio, norm="forward")
frequency_spectrum =  np.fft.rfftfreq(len(audio), 1 / sample_rate)

highpass_song = highpass_filter(frequency_spectrum,fourier_coefficients,3000)

plt.figure(figsize=(20, 5))
plt.plot(frequency_spectrum,highpass_song)
plt.xlabel("Frequency (Hz)", fontsize=14)

Audio(np.fft.irfft(highpass_song), rate=sample_rate)


In [ ]:
fourier_coefficients = np.fft.rfft(audio, norm="forward")
frequency_spectrum =  np.fft.rfftfreq(len(audio), 1 / sample_rate)

bandpass_song = bandpass_filter(frequency_spectrum,fourier_coefficients,1500,2000)

plt.figure(figsize=(20, 5))
plt.plot(frequency_spectrum,bandpass_song)
plt.xlabel("Frequency (Hz)", fontsize=14)

Audio(np.fft.irfft(bandpass_song), rate=sample_rate)

In [ ]:
fourier_coefficients = np.fft.rfft(audio, norm="forward")
frequency_spectrum =  np.fft.rfftfreq(len(audio), 1 / sample_rate)

bandstop_song = bandstop_filter(frequency_spectrum,fourier_coefficients,50,3000)

plt.figure(figsize=(20, 5))
plt.plot(frequency_spectrum,bandstop_song)
plt.xlabel("Frequency (Hz)", fontsize=14)

Audio(np.fft.irfft(bandstop_song), rate=sample_rate)

We've computed the Fourier coefficients, and manipulated them in some way. Now, we want to get back to audio. How can we do that? Think about how this would work for the Fourier series as we saw in class, and explore the NumPy documentation to discover how to do this.

Once you can get back to audio, try and modify your music sample in the following way:
 - remove the drums (to the best of your ability)
 - remove only the bass
 - extract only certain musical notes or chords (e.g., keep only "C" and its harmonics)
 - extract the part of only a certain instrument or set of instruments

In [ ]:
noSnare = bandstop_filter(frequency_spectrum,fourier_coefficients,200,250)
noDrums = bandstop_filter(frequency_spectrum,noSnare,30,60)

Audio(np.fft.irfft(noDrums), rate=sample_rate)

In [ ]:
noBass = bandstop_filter(frequency_spectrum,fourier_coefficients,60,250)

Audio(np.fft.irfft(noBass), rate=sample_rate)

In [ ]:
noD = bandstop_filter(frequency_spectrum,fourier_coefficients,293,295)

Audio(np.fft.irfft(noD), rate=sample_rate)

**Question 3**: How well did your simple filters work? Which of these tasks was (or do you think would be) the most difficult, and why?

The filters worked very well in reducing unwanted frequencies to zero. These are very idealized filters so they work as expected.

The bandstop filter was the most difficult to implement since it removes two different ranges of frequencies.

## Part 4: frequency-shaping filters and equalization

You've probably all played with something like [this](https://i0.wp.com/thetechhacker.com/wp-content/uploads/2016/10/FLAC-Player-SD-for-Windows-Phone-8.jpg?fit=472%2C787&ssl=1) before when attempting to improve the quality of sound coming through your computer or headphones. Rather than simply removing or extracting certain frequencies, sometimes we want to adjust the relative strengths of certain frequency bands. This could be for numerous reasons: optimizing how audio sounds in a particular room or physical space, emphasizing certain features depending on the musical genre, or balancing out the relative volumes of different instruments to highlight, e.g., a solo.  


Frequency bands are classified as bass, midrange, and high end (treble), often with further subdivisions therein. For example, here are a couple different charts that show this breakdown, along with rough frequency ranges:
 - https://www.teachmeaudio.com/mixing/techniques/audio-spectrum
 - https://www.sandburgmusic.org/uploads/4/6/7/1/46719067/editor/audiospectrum_1.gif
 - https://reference-audio-analyzer.pro/en/hp-fr.php#gsc.tab=0
 - https://www.masteringthemix.com/blogs/learn/understanding-the-different-frequency-ranges

In music equalization, when we seek to change the amplitudes of certain frequency bands, we are adjusting their *relative* volume, rather than an absolute increase or decrease in the amplitude. The unit of adjustment is the decibel (dB):

$$
\hbox{dB} = 20 \log_{10} \frac{\hbox{|Amplitude|}}{\hbox{|Reference amplitude|}}
$$

Note that the dB scale is logarithmic. For example, if we wanted to increase the strength of a frequency by 2 dB, we would need to rescale it by

$$
\hbox{|Amplitude|} \approx 1.26 \cdot \hbox{|Reference amplitude|}
$$

First, write a function to boost the bass of your signal by a specified number of dB.

In [ ]:
def bassboost(input_audio, bass):
    """Boost the bass of a given input signal.

    This function should do it all - compute the spectrum of the signal,
    adjust it as needed, and return the processed signal.

    Args:
        input_audio (array[float]): The input *audio* signal.
        bass (float): The amount, in dB, to boost the bass.

    Returns:
        array[float]: The output audio signal with boosted bass.
    """
    fourier_coefficients = np.fft.rfft(input_audio, norm="forward")
    frequencies =  np.fft.rfftfreq(len(input_audio), 1 / sample_rate)

    boost_factor = 10**(bass/20)

    boosted = []
    for i in range (0, len(frequencies)):
      if (frequencies[i] < 250 and frequencies[i] > 30):
        boosted.append(boost_factor*fourier_coefficients[i])
      else:
        boosted.append(fourier_coefficients[i])

    return boosted

Try applying it to your audio - does it sound like you expect?

In [ ]:
boosted = bassboost(audio, 10)
plt.figure(figsize=(20, 15))
plt.plot(boosted)
Audio(np.fft.irfft(boosted), rate=sample_rate)

In [ ]:
boosted = bassboost(audio, 5)
plt.figure(figsize=(20, 5))
plt.plot(boosted)
Audio(np.fft.irfft(boosted), rate=sample_rate)

**Question 4a**: After how many decibels of increase do you begin to actually hear a difference in the sound of the music? What do you notice about how it sounds now as a whole?

After about a 5 dB increase, we start to hear a more pronounced increase in the bass sound. The higher frequencies are not as loud by comparison.

Finally, write a more sophisticated equalizer that will allow you to adjust the strengths of multiple frequency bands. This is like a very simple programmatic version of an equalizer with multiple knobs or sliders.

Here are a few additional resources on audio mixing and equalization that you might find helpful:
 - https://www.teachmeaudio.com/mixing/techniques/overview
 - https://ampedstudio.com/equalization/
 - https://www.digitaltrends.com/home-theater/eq-explainer/
 - https://mynewmicrophone.com/complete-guide-to-audio-equalization-eq-hardware-software/

In [ ]:
def equalizer(input_audio, bass, mid, high):
    """Adjust the strengths of different frequency bands in an audio signal.

    You may use the original specifciation of bass, mid, high (and select the
    appropriate frequencies), or choose to make this more fine-grained.

    This function should do it all - compute the spectrum of the signal,
    adjust it as needed, and return the processed signal.

    Args:
        input_audio (array[float]): The input *audio* signal.
        bass (float): The amount, in dB, to adjust the bass frequency band.
        mid (float): The amount, in dB, to adjust the mid frequency band.
        high (float): The amount, in dB, to adjust the high frequency band.

    Returns:
        array[float]: The output audio signal with its frequency bands
        adjusted by the specified relative amplitudes.
    """
    fourier_coefficients = np.fft.rfft(input_audio, norm="forward")
    frequencies =  np.fft.rfftfreq(len(input_audio), 1 / sample_rate)

    bass_factor = 10**(bass/20)
    mid_factor = 10**(mid/20)
    high_factor = 10**(high/20)

    equalized = []
    for i in range (0, len(frequencies)):
      if (frequencies[i] < 250 and frequencies[i] > 1):
        equalized.append(bass_factor*fourier_coefficients[i])
      elif (frequencies[i] < 2000 and frequencies[i] >= 250):
        equalized.append(mid_factor*fourier_coefficients[i])
      elif (frequencies[i] >= 2000):
        equalized.append(high_factor*fourier_coefficients[i])
      else:
        equalized.append(fourier_coefficients[i])

    return equalized

In [ ]:
equalized = equalizer(audio, 10, -5, 20)
plt.figure(figsize=(20, 5))
plt.plot(equalized)
Audio(np.fft.irfft(equalized), rate=sample_rate)

Try out some different settings on your song!

In [ ]:
equalized = equalizer(audio, -10, 10, -10)
plt.figure(figsize=(20, 5))
plt.plot(equalized)
Audio(np.fft.irfft(equalized), rate=sample_rate)

In [ ]:
equalized = equalizer(audio, -20, -20, 50)
plt.figure(figsize=(20, 5))
plt.plot(equalized)
Audio(np.fft.irfft(equalized), rate=sample_rate)

**Question 4b:** Everything we have done so far has been on a relatively long clip of a song after it is loaded in. You might be wondering how all of this works in "real time", when you change the equalizer settings on the fly. How do you think audio signals are actually processed in software, e.g., when you're streaming a song?

There's a loop that checks and updates the equalizer settings each discretized time interval to update the discrete fourier coefficients for that time.

## Part 5: additional challenges

If you (optionally) want to take things further, try doing the following:
 - changing the musical key of your audio sample
 - playing with some more sophisticated filters (e.g., adding a slope with attenuation rather than a simple cutoff)
 - implement some "preset" equalization settings for specific genres of music

In [ ]:
def fancy_pass(input_audio, min_freq, max_freq, fade = False, bandpass = False):
  """ Can be used as either a bandpass or bandstop filter (defaults as bandstop)
  but instead of removing a frequency outside the specified range, it slowly
  attentuates it from the song. Ie: min_freq = 1000, you will still hear bass at
  the beginning but won't hear bass at the end of the song

  Args:
        input_audio (array[float]): The input *audio* signal.
        min_freq (float): Lowest frequency, in Hz, within the pass.
        mid (float): Highest frequency, in Hz, within the pass.
        fade (boolean): Whether to cut the pass or attentuate it over time, defaults false.
        bandpass (boolean): whether to act like a bandpass or bandstop, defaults false (bandstop)

    Returns:
        array[float]: The output audio signal with its frequency bands
        adjusted by the attenuation filter.

  """



  spectrum = np.fft.rfft(input_audio, norm="forward")
  frequencies =  np.fft.rfftfreq(len(input_audio), 1 / sample_rate)

  output = []
  print(spectrum)

  if bandpass:
    for i in range (0, len(frequencies)):
      if frequencies[i] < max_freq and frequencies[i] > min_freq:
        output.append(spectrum[i])
      else:
        output.append(spectrum[i]/(0.1*i))
  else:
    for i in range (0, len(frequencies)):
      if not (frequencies[i] < max_freq and frequencies[i] > min_freq):
        output.append(spectrum[i])
      else:
        output.append(spectrum[i]/(0.0000001*i))

  return output


In [ ]:
#You can hear the bass leaving the song
fancy = fancy_pass(audio, 1000, 5000, True, True)
plt.figure(figsize=(20, 5))
plt.plot(fancy)
Audio(np.fft.irfft(fancy), rate=sample_rate)

In [ ]:
#You can hear the bass leaving the song
fancy = fancy_pass(audio, 1, 5000, True, False)
plt.figure(figsize=(20, 5))
plt.plot(fancy)
Audio(np.fft.irfft(fancy), rate=sample_rate)